In [9]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# one review
train_data[1]

In [11]:
train_data = pad_sequences(train_data, MAXLEN)
test_data = pad_sequences(test_data, MAXLEN)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 76s 118ms/step - loss: 0.3584 - acc: 0.8487 - val_loss: 0.3485 - val_acc: 0.8516
Epoch 2/10
625/625 [==============================] - 77s 123ms/step - loss: 0.2363 - acc: 0.9099 - val_loss: 0.2907 - val_acc: 0.8822
Epoch 3/10
625/625 [==============================] - 75s 120ms/step - loss: 0.1879 - acc: 0.9324 - val_loss: 0.2911 - val_acc: 0.8796
Epoch 4/10
625/625 [==============================] - 74s 118ms/step - loss: 0.1535 - acc: 0.9458 - val_loss: 0.3217 - val_acc: 0.8780
Epoch 5/10
625/625 [==============================] - 72s 115ms/step - loss: 0.1244 - acc: 0.9575 - val_loss: 0.3692 - val_acc: 0.8760
Epoch 6/10
625/625 [==============================] - 75s 120ms/step - loss: 0.1044 - acc: 0.9663 - val_loss: 0.3671 - val_acc: 0.8730
Epoch 7/10
625/625 [==============================] - 78s 125ms/step - loss: 0.0862 - acc: 0.9725 - val_loss: 0.3929 - val_acc: 0.8746
Epoch 8/10
625/625 [==============================] - 7

In [16]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 25s 31ms/step - loss: 0.5979 - acc: 0.8422
[0.5979055166244507, 0.8421599864959717]


In [17]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [18]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [19]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


1/1 [==============================] - 0s 497ms/step
[0.9416429]
1/1 [==============================] - 0s 35ms/step
[0.06873361]
